# Import

In [1]:
import pandas as pd 
from urllib.request import urlopen
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time
import pprint

# Access API

In [2]:
response = urlopen('https://api.openf1.org/v1/sessions?session_name=Race&year=2023')
data = json.loads(response.read().decode('utf-8'))
df = pd.DataFrame(data)
response = urlopen('https://api.openf1.org/v1/sessions?session_name=Race&year=2024')
data = json.loads(response.read().decode('utf-8'))
df2 = pd.DataFrame(data)
df_sessions = pd.concat([df,df2])

# Create Key List

In [12]:
df_session_keys = df_sessions.drop(columns=['meeting_key', 'location', 'date_start', 'date_end','session_type', 'session_name', 'country_key', 'country_code','country_name', 'circuit_key', 'circuit_short_name', 'gmt_offset',
       'year'])

In [14]:
df_session_keys.to_csv('session_keys.csv', index=False)

# Upload raw_sessions to PSQL

In [24]:
df_sessions

,meeting_key,session_key,location,date_start,date_end,session_type,session_name,country_key,country_code,country_name,circuit_key,circuit_short_name,gmt_offset,year
0,1141,7953,Sakhir,2023-03-05T15:00:00+00:00,2023-03-05T17:00:00+00:00,Race,Race,36,BRN,Bahrain,63,Sakhir,03:00:00,2023
1,1142,7779,Jeddah,2023-03-19T17:00:00+00:00,2023-03-19T19:00:00+00:00,Race,Race,153,KSA,Saudi Arabia,149,Jeddah,03:00:00,2023
2,1143,7787,Melbourne,2023-04-02T05:00:00+00:00,2023-04-02T07:00:00+00:00,Race,Race,5,AUS,Australia,10,Melbourne,10:00:00,2023
3,1207,9070,Baku,2023-04-30T11:00:00+00:00,2023-04-30T13:00:00+00:00,Race,Race,30,AZE,Azerbaijan,144,Baku,04:00:00,2023
4,1208,9078,Miami,2023-05-07T19:30:00+00:00,2023-05-07T21:30:00+00:00,Race,Race,19,USA,United States,151,Miami,-04:00:00,2023
5,1210,9094,Monaco,2023-05-28T13:00:00+00:00,2023-05-28T15:00:00+00:00,Race,Race,114,MON,Monaco,22,Monte Carlo,02:00:00,2023
6,1211,9102,Barcelona,2023-06-04T13:00:00+00:00,2023-06-04T15:00:00+00:00,Race,Race,1,ESP,Spain,15,Catalunya,02:00:00,2023
7,1212,9110,Montréal,2023-06-18T18:00:00+00:00,2023-06-18T20:00:00+00:00,Race,Race,46,CAN,Canada,23,Montreal,-04:00:00,2023
8,1213,9118,Spielberg,2023-07-02T13:00:00+00:00,2023-07-02T15:00:00+00:00,Race,Race,17,AUT,Austria,19,Spielberg,02:00:00,2023
9,1214,9126,Silverstone,2023-07-09T14:00:00+00:00,2023-07-09T16:00:00+00:00,Race,Race,2,GBR,Great Britain,2,Silverstone,01:00:00,2023


In [15]:
df_sessions.columns

Index(['meeting_key', 'session_key', 'location', 'date_start', 'date_end',
       'session_type', 'session_name', 'country_key', 'country_code',
       'country_name', 'circuit_key', 'circuit_short_name', 'gmt_offset',
       'year'],
      dtype='object')

In [26]:
config = dotenv_values()

pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [27]:
# defining data types for the DB
dtype_dict = {
    'meeting_key': types.Integer,
    'session_key': types.Integer,
    'location': types.String,
    'date_start': types.DateTime,
    'date_end': types.DateTime,
    'session_type': types.String,
    'session_name': types.String,
    'country_key': types.Integer,
    'country_code': types.String,
    'country_name': types.String,
    'circuit_key': types.Integer,
    'circuit_short_name': types.String,
    'gmt_offset': types.String,        # could be Interval if stored as "+02:00"
    'year': types.Integer
}

In [28]:
df_sessions.to_sql(name = 'raw_sessions', 
                       con = engine, 
                       schema = pg_schema, 
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

46